# Create Postgres Database from CVE data

In [5]:
import os
import json
import psycopg2
from packaging.version import Version, InvalidVersion

# PostgreSQL database connection details
db_config = {
    "dbname": "cve_database",
    "user": "postgres",
    "password": "ionut",  # Add password if needed
    "host": "localhost",
    "port": 5432
}

# Connect to PostgreSQL
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

# Step 1: Define the database schema
cursor.execute("""
CREATE TABLE IF NOT EXISTS cpe (
    cpe_id TEXT PRIMARY KEY
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS cve (
    cpe_id TEXT,
    cve_id TEXT,
    cve_data JSONB,
    PRIMARY KEY (cpe_id, cve_id),
    FOREIGN KEY (cpe_id) REFERENCES cpe (cpe_id)
)
""")
conn.commit()

# Step 2: Process JSON files
def process_json(file_path):
    """Extract CPE and CVE information from a JSON file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    cpe_cve_map = {}
    for item in data.get("CVE_Items", []):
        cve_id = item.get("cve", {}).get("CVE_data_meta", {}).get("ID", "Unknown")
        cve_full_data = json.dumps(item)  # Store the entire CVE object

        # Extract CPEs
        configurations = item.get("configurations", {}).get("nodes", [])
        for node in configurations:
            for cpe in node.get("cpe_match", []):
                cpe_uri = cpe.get("cpe23Uri")
                if cpe_uri:
                    if cpe_uri not in cpe_cve_map:
                        cpe_cve_map[cpe_uri] = []
                    # Add only unique CVE IDs for each CPE
                    if cve_id not in [entry["cve_id"] for entry in cpe_cve_map[cpe_uri]]:
                        cpe_cve_map[cpe_uri].append({
                            "cve_id": cve_id,
                            "cve_data": cve_full_data
                        })
    return cpe_cve_map

def save_to_database(cpe_cve_map):
    """Save extracted CPE-CVE mappings to the PostgreSQL database."""
    for cpe, cves in cpe_cve_map.items():
        # Insert the CPE into the cpe table
        cursor.execute("""
        INSERT INTO cpe (cpe_id)
        VALUES (%s)
        ON CONFLICT (cpe_id) DO NOTHING
        """, (cpe,))
        
        # Insert the CVEs into the cve table
        for cve_entry in cves:
            cursor.execute("""
            INSERT INTO cve (cpe_id, cve_id, cve_data)
            VALUES (%s, %s, %s)
            ON CONFLICT (cpe_id, cve_id) DO NOTHING
            """, (cpe, cve_entry["cve_id"], cve_entry["cve_data"]))
    conn.commit()

# Step 3: Iterate through all JSON files
base_dir = "../data/CVEs"  # Directory containing CVE JSON files
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".json"):
            file_path = os.path.join(root, file)
            print(f"Processing file: {file_path}")
            cpe_cve_map = process_json(file_path)
            save_to_database(cpe_cve_map)

# Step 4: Close the database connection
cursor.close()
conn.close()
print("Database created and populated successfully!")


Processing file: ../data/CVEs/2009/nvdcve-1.1-2009.json
Processing file: ../data/CVEs/modified/nvdcve-1.1-modified.json
Processing file: ../data/CVEs/2022/nvdcve-1.1-2022.json
Processing file: ../data/CVEs/2010/nvdcve-1.1-2010.json
Processing file: ../data/CVEs/2020/nvdcve-1.1-2020.json
Processing file: ../data/CVEs/2011/nvdcve-1.1-2011.json
Processing file: ../data/CVEs/2008/nvdcve-1.1-2008.json
Processing file: ../data/CVEs/2005/nvdcve-1.1-2005.json
Processing file: ../data/CVEs/2002/nvdcve-1.1-2002.json
Processing file: ../data/CVEs/2013/nvdcve-1.1-2013.json
Processing file: ../data/CVEs/2015/nvdcve-1.1-2015.json
Processing file: ../data/CVEs/2003/nvdcve-1.1-2003.json
Processing file: ../data/CVEs/2024/nvdcve-1.1-2024.json
Processing file: ../data/CVEs/2014/nvdcve-1.1-2014.json
Processing file: ../data/CVEs/2006/nvdcve-1.1-2006.json
Processing file: ../data/CVEs/2007/nvdcve-1.1-2007.json
Processing file: ../data/CVEs/2021/nvdcve-1.1-2021.json
Processing file: ../data/CVEs/2012/nvdcv

# Query to test the database

In [9]:
import psycopg2

# PostgreSQL database connection details
db_config = {
    "dbname": "cve_database",  
    "user": "postgres",        
    "password": "ionut", 
    "host": "localhost",       
    "port": 5432              
}

# Define the CPE to search for
cpe = 'cpe:2.3:a:solarwinds:serv-u:*:*:*:*:*:*:*:*'

# Connect to the PostgreSQL database
try:
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()

    # Extract the prefix from the CPE (up to the last colon)
    cpe_prefix = f"{':'.join(cpe.split(':')[:5])}:"

    # Query to fetch matching data
    query = """
    SELECT cve_id, cve_data
    FROM cve
    WHERE cpe_id LIKE %s
    """
    
    # Execute the query with the CPE prefix
    cursor.execute(query, (f"{cpe_prefix}%",))
    results = cursor.fetchall()

    # Display the results
    if results:
        print(f"Matching CVEs for CPE '{cpe}':")
        for cve_id, cve_data in results:
            print(f"- CVE ID: {cve_id}")
            print(f"  CVE Data: {cve_data}\n")
    else:
        print(f"No matching CVEs found for CPE '{cpe}'.")

except psycopg2.Error as e:
    print(f"Database error: {e}")
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()


Matching CVEs for CPE 'cpe:2.3:a:solarwinds:serv-u:*:*:*:*:*:*:*:*':
- CVE ID: CVE-2021-35211
  CVE Data: {'cve': {'data_type': 'CVE', 'references': {'reference_data': [{'url': 'https://www.microsoft.com/security/blog/2021/07/13/microsoft-discovers-threat-actor-targeting-solarwinds-serv-u-software-with-0-day-exploit', 'name': 'https://www.microsoft.com/security/blog/2021/07/13/microsoft-discovers-threat-actor-targeting-solarwinds-serv-u-software-with-0-day-exploit', 'tags': ['Patch', 'Vendor Advisory'], 'refsource': ''}, {'url': 'https://www.microsoft.com/security/blog/2021/07/13/microsoft-discovers-threat-actor-targeting-solarwinds-serv-u-software-with-0-day-exploit', 'name': 'https://www.microsoft.com/security/blog/2021/07/13/microsoft-discovers-threat-actor-targeting-solarwinds-serv-u-software-with-0-day-exploit', 'tags': ['Patch', 'Vendor Advisory'], 'refsource': ''}, {'url': 'https://www.solarwinds.com/trust-center/security-advisories/cve-2021-35211', 'name': 'https://www.solarwin

# UPDATE DATABASE WITH IN BETWEEN CVES

In [8]:
import os
import json
import psycopg2
from packaging.version import Version, InvalidVersion

# PostgreSQL database connection details
db_config = {
    "dbname": "cve_database",
    "user": "postgres",
    "password": "ionut",
    "host": "localhost",
    "port": 5432
}

# Connect to PostgreSQL
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

def parse_version(version_str):
    """Parse a version string into a comparable format. Return None if invalid."""
    try:
        return Version(version_str)
    except InvalidVersion:
        return None

def is_version_in_range(version, start_including=None, end_excluding=None):
    """Check if a version is within a given range."""
    version_obj = parse_version(version)
    if not version_obj:
        return False

    if start_including:
        start_version = parse_version(start_including)
        if start_version and version_obj < start_version:
            return False
    if end_excluding:
        end_version = parse_version(end_excluding)
        if end_version and version_obj >= end_version:
            return False
    return True

def process_json_for_version_ranges(file_path):
    """Identify CVEs with version ranges and update the PostgreSQL database."""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for item in data.get("CVE_Items", []):
        cve_id = item.get("cve", {}).get("CVE_data_meta", {}).get("ID", "Unknown")
        full_cve_data = json.dumps(item)  # Store the full CVE object

        configurations = item.get("configurations", {}).get("nodes", [])
        for node in configurations:
            for cpe in node.get("cpe_match", []):
                cpe_uri = cpe.get("cpe23Uri")
                version_start = cpe.get("versionStartIncluding")
                version_end = cpe.get("versionEndExcluding")

                if cpe_uri and (version_start or version_end):
                    # Extract the base CPE name
                    base_cpe = ":".join(cpe_uri.split(":")[:5])

                    # Query the database for all matching CPEs
                    cursor.execute("SELECT cpe_id FROM cpe WHERE cpe_id LIKE %s", (f"{base_cpe}%",))
                    matching_cpes = cursor.fetchall()

                    for matching_cpe_row in matching_cpes:
                        matching_cpe = matching_cpe_row[0]
                        # Extract the version from the matching CPE
                        cpe_parts = matching_cpe.split(":")
                        if len(cpe_parts) > 5:  # Ensure version exists
                            version = cpe_parts[5]
                            if is_version_in_range(version, version_start, version_end):
                                # Add the CVE to the database for this CPE
                                cursor.execute("""
                                INSERT INTO cve (cpe_id, cve_id, cve_data)
                                VALUES (%s, %s, %s)
                                ON CONFLICT (cpe_id, cve_id) DO NOTHING
                                """, (matching_cpe, cve_id, full_cve_data))
    conn.commit()

# Process JSON files for version ranges
base_dir = "../data/CVEs"
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".json"):
            file_path = os.path.join(root, file)
            print(f"Processing file for version ranges: {file_path}")
            process_json_for_version_ranges(file_path)

# Close the database connection
cursor.close()
conn.close()
print("Database updated successfully!")


Processing file for version ranges: ../data/CVEs/2009/nvdcve-1.1-2009.json
Processing file for version ranges: ../data/CVEs/modified/nvdcve-1.1-modified.json
Processing file for version ranges: ../data/CVEs/2022/nvdcve-1.1-2022.json
Processing file for version ranges: ../data/CVEs/2010/nvdcve-1.1-2010.json
Processing file for version ranges: ../data/CVEs/2020/nvdcve-1.1-2020.json
Processing file for version ranges: ../data/CVEs/2011/nvdcve-1.1-2011.json
Processing file for version ranges: ../data/CVEs/2008/nvdcve-1.1-2008.json
Processing file for version ranges: ../data/CVEs/2005/nvdcve-1.1-2005.json
Processing file for version ranges: ../data/CVEs/2002/nvdcve-1.1-2002.json
Processing file for version ranges: ../data/CVEs/2013/nvdcve-1.1-2013.json
Processing file for version ranges: ../data/CVEs/2015/nvdcve-1.1-2015.json
Processing file for version ranges: ../data/CVEs/2003/nvdcve-1.1-2003.json
Processing file for version ranges: ../data/CVEs/2024/nvdcve-1.1-2024.json
Processing file f